# Image Deconvolution
- ver.0.0.2
- 2023 May 4
- H. Yoneda
- using 1s GRB simulation data from source_injector.ipynb
- sc_orientation is considered
- parameters can be changed

- Adapted 2024 by E. Shread

In [1]:
import astropy.units as u
from astropy.time import Time

import matplotlib.pyplot as plt
import numpy as np

from mhealpy import HealpixMap
from scoords import SpacecraftFrame, Attitude
from cosipy.data_io import DataIO
from cosipy.image_deconvolution import DataLoader
from cosipy.image_deconvolution import ImageDeconvolution

import warnings
warnings.simplefilter("once")

03:00:31 WARNING   The naima package is not available. Models that depend on it will not be         ]8;id=217358;file:///home/eshread/.local/lib/python3.10/site-packages/astromodels/functions/functions_1D/functions.py\functions.py]8;;\:]8;id=262700;file:///home/eshread/.local/lib/python3.10/site-packages/astromodels/functions/functions_1D/functions.py#48\48]8;;\
                  available                                                                                        

         WARNING   The GSL library or the pygsl wrapper cannot be loaded. Models that depend on it  ]8;id=487746;file:///home/eshread/.local/lib/python3.10/site-packages/astromodels/functions/functions_1D/functions.py\functions.py]8;;\:]8;id=414956;file:///home/eshread/.local/lib/python3.10/site-packages/astromodels/functions/functions_1D/functions.py#69\69]8;;\
                  will not be available.                                                                           

/home/eshread/.local/lib/python3.10/site-packages/numba/core/decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)


03:00:32 WARNING   The ebltable package is not available. Models that depend on it will not be     ]8;id=968944;file:///home/eshread/.local/lib/python3.10/site-packages/astromodels/functions/functions_1D/absorption.py\absorption.py]8;;\:]8;id=639164;file:///home/eshread/.local/lib/python3.10/site-packages/astromodels/functions/functions_1D/absorption.py#36\36]8;;\
                  available                                                                                        

         WARNING   We have set the min_value of K to 1e-99 because there was a postive transform   ]8;id=784220;file:///home/eshread/.local/lib/python3.10/site-packages/astromodels/core/parameter.py\parameter.py]8;;\:]8;id=751092;file:///home/eshread/.local/lib/python3.10/site-packages/astromodels/core/parameter.py#704\704]8;;\

...

         WARNING   Could not import plugin FermiLATLike.py. Do you have the relative instrument     ]8;id=215662;file:///home/eshread/.local/lib/python3.10/site-packages/threeML/__init__.py\__init__.py]8;;\:]8;id=483436;file:///home/eshread/.local/lib/python3.10/site-packages/threeML/__init__.py#144\144]8;;\
                  software installed and configured?                                                               

03:00:34 WARNING   No fermitools installed                                              ]8;id=194438;file:///home/eshread/.local/lib/python3.10/site-packages/threeML/utils/data_builders/fermi/lat_transient_builder.py\lat_transient_builder.py]8;;\:]8;id=980381;file:///home/eshread/.local/lib/python3.10/site-packages/threeML/utils/data_builders/fermi/lat_transient_builder.py#44\44]8;;\

03:00:34 WARNING   Env. variable OMP_NUM_THREADS is not set. Please set it to 1 for optimal         ]8;id=870949;file:///home/eshread/.local/lib/python3.10/site-packages/threeML/__init__.py\__init__.py]8;;\:]8;id=539621;file:///home/eshread/.local/lib/python3.10/site-packages/threeML/__init__.py#387\387]8;;\
                  performances in 3ML                                                                              

         WARNING   Env. variable MKL_NUM_THREADS is not set. Please set it to 1 for optimal         ]8;id=389343;file:///home/eshread/.local/lib/python3.10/site-packages/threeML/__init__.py\__init__.py]8;;\:]8;id=935110;file:///home/eshread/.local/lib/python3.10/site-packages/threeML/__init__.py#387\387]8;;\
                  performances in 3ML                                                                              

         WARNING   Env. variable NUMEXPR_NUM_THREADS is not set. Please set it to 1 for optimal     ]8;id=934685;file:///home/eshread/.local/lib/python3.10/site-packages/threeML/__init__.py\__init__.py]8;;\:]8;id=142310;file:///home/eshread/.local/lib/python3.10/site-packages/threeML/__init__.py#387\387]8;;\
                  performances in 3ML                                                                              

In [2]:
event_tra_filepath = '/home/eshread/nn-response-approximation/TestSource.520.inc1.id1.tra.gz'

event_unbinned_yaml_filepath = '/home/eshread/nn-response-approximation/tra_events.yaml'

#uncomment for option to bin data directly from specifications of a yaml file
#event_binned_yaml_filepath = '/home/eshread/nn-response-approximation/binned_events.yaml'

response_filepath = '/home/eshread/nn-response-approximation/cross-sec-data/FlatContinuumIsotropic.inc520.id1.tra.gz.pkl'

## load files

- Need the dataIO library for the event and response files

In [3]:
dataloader = DataLoader.load_from_filepath(event_tra_filepath = event_tra_filepath, event_unbinned_yaml_filepath = event_unbinned_yaml_filepath, 
                                        rsp_filepath = response_filepath)

... loading event from /home/eshread/nn-response-approximation/TestSource.520.inc1.id1.tra.gz and /home/eshread/nn-response-approximation/tra_events.yaml
Preparing to read file...
Reading file...


  0%|                                                                                   | 0/132254619.0 [00:00<?, ?it/s]
WARNING RuntimeWarning: invalid value encountered in arccos

  0%|                                                                   | 13213/132254619.0 [00:00<06:29, 339219.69it/s]

Stopping here: only reading a subset of events
Making COSI data set...
Saving file...
... Done ...
... loading full detector response from /home/eshread/nn-response-approximation/cross-sec-data/FlatContinuumIsotropic.inc520.id1.tra.gz.pkl
... Done ...


In [ ]:
dataloader.response.axes["Ei"].edges

In [ ]:
dataloader.response.axes.labels

In [ ]:
dataloader.response.axes['Phi'].widths

## load data

In [ ]:
image_deconvolution = ImageDeconvolution()

In [ ]:
image_deconvolution.set_data(dataloader)

## load parameterfile

In [ ]:
parameter_filepath = test_data.path / "imagedeconvolution_parfile_itr10.yml"

image_deconvolution.read_parameterfile(parameter_filepath)

## sparse matrix?

In [ ]:
image_deconvolution.use_sparse = False

## initialization
- produce an initial flux map etc.

In [ ]:
image_deconvolution.initialize()

## change the parameter
- change the iteration from 10 to 20
- you should run "initialize" after parameters are changed

In [ ]:
print(image_deconvolution.parameter.dump())

In [ ]:
image_deconvolution.override_parameter("deconvolution:parameter_RL:iteration = 20")

In [ ]:
print(image_deconvolution.parameter.dump())

In [ ]:
image_deconvolution.initialize()

## run deconvolution

In [ ]:
%%time

all_result = image_deconvolution.run_deconvolution()

## analyze the result


In [ ]:
print(all_result)

### likelihood

In [ ]:
x, y = [], []

for _ in all_result:
    x.append(_['iteration'])
    y.append(_['loglikelihood'])
    
plt.plot(x, y)
plt.xlabel("iteration")
plt.ylabel("loglikelihood")

### alpha

In [ ]:
x, y = [], []

for _ in all_result:
    x.append(_['iteration'])
    y.append(_['alpha'])
    
plt.plot(x, y)
plt.xlabel("iteration")
plt.ylabel("alpha")

### image

In [ ]:
for _ in all_result:
    iteration = _['iteration']
    image = _['model_map']
        
    for energy_band in range(5):
        map_healpxmap = HealpixMap(data = image[:,energy_band], unit = image.unit)

        map_healpxmap.plot()
        plt.title(label = "iteration = {}, energyband = {}".format(iteration, energy_band))
        plt.show()

### delta_image

In [ ]:
for _ in all_result:
    iteration = _['iteration']
    image = _['delta_map']
        
    for energy_band in range(5):
        map_healpxmap = HealpixMap(data = image[:,energy_band], unit = image.unit)

        map_healpxmap.plot()
        plt.title(label = "iteration = {}, energyband = {}".format(iteration, energy_band))
        plt.show()

### flux

In [ ]:
iteration = []
integrated_flux = []
integrated_flux_each_band = [[],[],[],[],[]]

for _ in all_result:
    iteration.append(_['iteration'])
    image = _['model_map']

    integrated_flux.append( np.sum(image) / image.unit)

    for energy_band in range(5):
        integrated_flux_each_band[energy_band].append( np.sum(image[:,energy_band]) / image.unit)
        
plt.plot(iteration, integrated_flux, label = 'total')
plt.xlabel("iteration")
plt.ylabel("integrated flux")
plt.yscale("log")

for energy_band in range(5):
    plt.plot(iteration, integrated_flux_each_band[energy_band], label = "energyband = {}".format(energy_band))
    
plt.legend()

### flux delta

In [ ]:
iteration = []
delta_integrated_flux = []
delta_integrated_flux_each_band = [[],[],[],[],[]]

for _ in all_result:
    iteration.append(_['iteration'])
    delta_image = _['delta_map']

    delta_integrated_flux.append( np.sum(delta_image) / delta_image.unit)

    for energy_band in range(5):
        delta_integrated_flux_each_band[energy_band].append( np.sum(delta_image[:,energy_band]) / delta_image.unit)
        
plt.plot(iteration, delta_integrated_flux, label = 'total')
plt.xlabel("iteration")
plt.ylabel("delta integrated flux")
plt.ylim(-0.08, 0.08)

for energy_band in range(5):
    plt.plot(iteration, delta_integrated_flux_each_band[energy_band], label = "energyband = {}".format(energy_band))
    
plt.legend()

## comparision with the truth value

In [ ]:
from astromodels import Powerlaw
from scipy import integrate
from astropy.units import Quantity

spectrum = Powerlaw(K = 1e-1, index = -1.7)

integrated_flux_truth = integrate.quad(spectrum, 100, 5000.0)[0] / u.cm**2 / u.s
pixel_area = 4 * 3.1415 * u.sr / all_result[0]['model_map'].axes['NuLambda'].npix

print("truth:", integrated_flux_truth)

print("deconvolution:", integrated_flux[-1] * pixel_area * u.cm**-2 * u.s**-1 * u.sr**-1)

In [ ]:
integrated_flux_each_band_truth = []

for energy_band in range(5):
    emin, emax = dataloader.response.axes["Ei"].bounds[energy_band]

    integrated_flux_each_band_truth.append(integrate.quad(spectrum, emin.value, emax.value)[0] / u.cm**2 / u.s)
    pixel_area = 4 * 3.1415 * u.sr / all_result[0]['model_map'].axes['NuLambda'].npix
    
    print(emin, emax)
    print("    truth:", integrated_flux_each_band_truth[energy_band])
    print("    deconvolution:", integrated_flux_each_band[energy_band][-1] * pixel_area * u.cm**-2 * u.s**-1 * u.sr**-1)

In [ ]:
iteration = []
integrated_flux = []
integrated_flux_each_band = [[],[],[],[],[]]

for _ in all_result:
    iteration.append(_['iteration'])
    image = _['model_map']

    integrated_flux.append( np.sum(image) / image.unit)

    for energy_band in range(5):
        integrated_flux_each_band[energy_band].append( np.sum(image[:,energy_band]) / image.unit)
        
plt.plot(iteration, integrated_flux * pixel_area, label = 'total', color = 'black')
plt.plot(iteration, np.full(len(iteration), integrated_flux_truth), color = 'black', linestyle = "--")
plt.xlabel("iteration")
plt.ylabel("integrated flux")
plt.yscale("log")


colors = ['b', 'g', 'r', 'c', 'm']
for energy_band in range(5):
    plt.plot(iteration, integrated_flux_each_band[energy_band] * pixel_area, label = "energyband = {}".format(energy_band), color = colors[energy_band])
    plt.plot(iteration, np.full(len(iteration), integrated_flux_each_band_truth[energy_band]), color = colors[energy_band], linestyle = "--")
    
plt.legend()